# Non supervised text topic detection

## LDA with Gensim

In [13]:
import os
from glob import glob
import re
import string
import funcy as fp

import pandas as pd

data = '../data'

In [2]:
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
print(len(twenty_train['data']))

11314


In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer

def tokenize_doc(text):
    sents = sent_tokenize(text)
    tokens = []
    for s in sents:
        for t in TreebankWordTokenizer().tokenize(s):
            if len(t)<2 or '\n' in t or '\t' in t or t.isnumeric():
                continue
            else:
                tokens += [t.lower()]
    return tokens

In [5]:
%%time

docs_tokenized = []
for text in twenty_train['data']:
    docs_tokenized += [tokenize_doc(text)]
print(len(docs_tokenized))

11314
CPU times: user 22.6 s, sys: 113 ms, total: 22.7 s
Wall time: 22.7 s


In [6]:
# Get the NLTK stopwords list
import nltk

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

In [7]:
# create dictionary and bag of words corpus

from gensim.corpora import Dictionary, MmCorpus
    
print('Building dictionary...')
dictionary = Dictionary(docs_tokenized)

# Stopwords
additional_stopwords=set(['...', '\'s','\'\'',  "``", "n\'t", "\'re", "\'m", "\'ve",
                          '--', '"', "\'ax", 'max', 'q,3'])

stopwords = nltk_stopwords().union(additional_stopwords)
stopword_ids = map(dictionary.token2id.get, stopwords)
dictionary.filter_tokens(stopword_ids)
dictionary.compactify()

# Filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=None)
dictionary.compactify()

print('Building corpus...')
corpus = [dictionary.doc2bow(doc) for doc in docs_tokenized]

Building dictionary...
Building corpus...


In [14]:
%%time

# Train the LDA model

from gensim import models

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50, passes=10)
                                      
lda.save(os.path.join(data,'newsgroups_50_lda.model'))

CPU times: user 1min 25s, sys: 1.43 s, total: 1min 26s
Wall time: 1min 25s


In [15]:
lda.show_topics()

[(2,
  '0.019*"new" + 0.019*"apple" + 0.015*"cd" + 0.015*"shipping" + 0.013*"cover" + 0.013*"dos" + 0.012*"copy" + 0.011*"appears" + 0.011*"copies" + 0.011*"price"'),
 (35,
  '0.021*"book" + 0.021*"jumper" + 0.020*"myers" + 0.019*".." + 0.017*"manual" + 0.015*"reference" + 0.012*"turbo" + 0.012*"de" + 0.011*"ms" + 0.010*"page"'),
 (28,
  '0.018*"trial" + 0.013*"dog" + 0.013*"news" + 0.012*"new" + 0.012*"media" + 0.010*"joseph" + 0.009*"washington" + 0.009*"authorities" + 0.008*"court" + 0.008*"attorney"'),
 (26,
  '0.037*"gun" + 0.020*"guns" + 0.019*"crime" + 0.018*"control" + 0.016*"police" + 0.013*"weapon" + 0.013*"firearms" + 0.012*"weapons" + 0.011*"firearm" + 0.010*"law"'),
 (8,
  '0.031*"team" + 0.031*"game" + 0.023*"games" + 0.021*"hockey" + 0.020*"play" + 0.017*"season" + 0.016*"league" + 0.013*"san" + 0.013*"nhl" + 0.012*"period"'),
 (41,
  '0.020*"car" + 0.013*"price" + 0.013*"new" + 0.011*"get" + 0.009*"like" + 0.009*"bike" + 0.008*"used" + 0.008*"buy" + 0.008*"good" + 0.007

# Visualization

In [16]:
import pyLDAvis

pyLDAvis.enable_notebook()

In [17]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

/Users/jorge/anaconda3/envs/tm/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
25     16.133837        1       1  0.255322 -0.124616
11      6.040994        1       2  0.200167 -0.103672
23      5.156169        1       3  0.066848 -0.161091
19      4.707445        1       4  0.225137  0.114987
30      3.455264        1       5  0.179714  0.051626
41      3.348857        1       6  0.170988  0.023121
48      3.337788        1       7  0.146001 -0.189066
7       3.270938        1       8  0.064459  0.141202
21      3.050772        1       9  0.169663 -0.127766
42      2.947618        1      10  0.127956  0.064629
12      2.799571        1      11  0.186514  0.009473
43      2.798822        1      12  0.053183  0.102309
36      2.642549        1      13  0.059918 -0.018013
33      2.583340        1      14  0.116465 -0.080573
27      2.551640        1      15  0.137406  0.130537
15      2.260097        1      16  0.088718 -0.125083
44      2.153104        1      17  0.069247  0.136638
8       1.984040        1      18 -0.117842 -0.090694
22      1.595003        1      19  0.046703  0.214139
26      1.534594        1      20 -0.013748 -0.080301
39      1.524211        1      21 -0.213343  0.012945
9       1.487554        1      22  0.029122  0.153874
34      1.454577        1      23  0.077637 -0.088358
17      1.428154        1      24  0.033704 -0.051743
20      1.413704        1      25 -0.215172  0.024853
4       1.369776        1      26  0.066338  0.146908
46      1.216452        1      27 -0.057269 -0.093358
38      1.181119        1      28 -0.059614 -0.029399
32      1.160556        1      29  0.065474  0.115307
24      1.118076        1      30 -0.064404 -0.104509
37      1.070481        1      31 -0.035980  0.160483
2       1.040217        1      32 -0.021728  0.137509
6       0.924453        1      33 -0.012115 -0.035294
16      0.917735        1      34 -0.044044 -0.128505
28      0.868722        1      35 -0.090443 -0.097481
14      0.717774        1      36 -0.076942 -0.115900
47      0.651692        1      37 -0.124877 -0.003516
18      0.625465        1      38  0.028386 -0.032545
35      0.622452        1      39 -0.054056  0.067591
13      0.567281        1      40 -0.202100  0.008612
10      0.558077        1      41 -0.155257  0.008596
3       0.549249        1      42 -0.092833  0.032048
0       0.548319        1      43 -0.058323 -0.031325
45      0.520843        1      44 -0.050642 -0.026376
5       0.505063        1      45 -0.098552  0.064326
40      0.405256        1      46 -0.141709  0.018207
31      0.375727        1      47 -0.106205  0.039128
29      0.287583        1      48 -0.193605 -0.012538
49      0.278136        1      49 -0.177484 -0.009955
1       0.258851        1      50 -0.186781 -0.017367, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
1360   Default  2116.000000          god  2116.000000  30.0000  30.0000
350    Default  6425.000000        would  6425.000000  29.0000  29.0000
257    Default  1505.000000         file  1505.000000  28.0000  28.0000
931    Default  1122.000000        drive  1122.000000  27.0000  27.0000
4609   Default   982.000000          b8f   982.000000  26.0000  26.0000
146    Default  4007.000000       people  4007.000000  25.0000  25.0000
599    Default  1098.000000          key  1098.000000  24.0000  24.0000
18     Default  3429.000000         know  3429.000000  23.0000  23.0000
669    Default  1056.000000        space  1056.000000  22.0000  22.0000
130    Default  3903.000000         like  3903.000000  21.0000  21.0000
4593   Default   809.000000          a86   809.000000  20.0000  20.0000
502    Default  6184.000000          one  6184.000000  19.0000  19.0000
120    Default  2391.000000         good  2391.000000  18.0000  18.0000
182    Default  3162.000000          use  3162.000000  17.0000  17.00